# Mathematical Optimisation - Project
This project aims at ricreating the results reported in:

[A fix-and-optimize heuristic for the Unrelated Parallel Machine Scheduling Problem
](https://www.sciencedirect.com/science/article/pii/S0305054823003684)

In [8]:
from Solver import Solver
from MPA.MPA import MPA
from Heuristic.Greedy import Greedy
from Heuristic.FixOpt import FixOpt
import random as r
import json
from IPython.display import clear_output

r.seed(3000)
  
with open('credentials.txt') as f: 
    data = f.read() 
options = json.loads(data)

## Creation of the matrices

- P stores the execution times
- S stores the setup times

These matrices are converted to dictionaries to handle the indexes not starting from 0

In [2]:
def generate_instance(N_cardinality, M_cardinality,
                      min_execution_time = 1,max_execution_time = 10,
                      min_setup_time = 1,max_setup_time = 3):
    
    P = {}
    S = {}
    
    N = range(1, N_cardinality+1)
    M = range(1, M_cardinality+1)
    
    for i in M:
        for j in N:
            P[i,j] = r.randint(min_execution_time, max_execution_time)
            
    for i in M:
        for j in N:
            for k in N:
                S[i,j,k] = r.randint(min_setup_time, max_setup_time)
    
    return P,S

N_cardinality = 30
M_cardinality = 5

P_dict, S_dict = generate_instance(N_cardinality=N_cardinality,M_cardinality=M_cardinality)
print(f'P: {P_dict}')
print(f'S: {S_dict}')

P: {(1, 1): 3, (1, 2): 5, (1, 3): 2, (1, 4): 5, (1, 5): 5, (1, 6): 3, (1, 7): 10, (1, 8): 8, (1, 9): 5, (1, 10): 4, (1, 11): 3, (1, 12): 9, (1, 13): 4, (1, 14): 4, (1, 15): 9, (1, 16): 6, (1, 17): 10, (1, 18): 10, (1, 19): 7, (1, 20): 7, (1, 21): 5, (1, 22): 9, (1, 23): 6, (1, 24): 7, (1, 25): 4, (1, 26): 7, (1, 27): 7, (1, 28): 4, (1, 29): 6, (1, 30): 6, (2, 1): 3, (2, 2): 3, (2, 3): 2, (2, 4): 9, (2, 5): 7, (2, 6): 8, (2, 7): 2, (2, 8): 2, (2, 9): 10, (2, 10): 10, (2, 11): 2, (2, 12): 9, (2, 13): 8, (2, 14): 3, (2, 15): 8, (2, 16): 8, (2, 17): 4, (2, 18): 5, (2, 19): 9, (2, 20): 2, (2, 21): 2, (2, 22): 4, (2, 23): 3, (2, 24): 7, (2, 25): 9, (2, 26): 3, (2, 27): 6, (2, 28): 10, (2, 29): 9, (2, 30): 4, (3, 1): 3, (3, 2): 6, (3, 3): 4, (3, 4): 4, (3, 5): 4, (3, 6): 9, (3, 7): 1, (3, 8): 10, (3, 9): 9, (3, 10): 7, (3, 11): 2, (3, 12): 4, (3, 13): 10, (3, 14): 9, (3, 15): 4, (3, 16): 3, (3, 17): 5, (3, 18): 1, (3, 19): 10, (3, 20): 1, (3, 21): 3, (3, 22): 5, (3, 23): 10, (3, 24): 7, (3, 2

In [3]:
N = range(1, N_cardinality+1)
M = range(1, M_cardinality+1)
N0 = [i for i in N]
N0.insert(0,0)

# Exact Solution

The problem is solved exactly using the Solver class.

Then, the results are printed. Note that job 0 is a dummy job used to represent the beginning and the end of activities for each machine.

In [4]:
s = Solver(execution_times = P_dict, setup_times = S_dict)
decision_variables,completion_times,maximum_makespan,assignments = s.solve(options=options)

for i in s.M:
    for j in s.N:
        for k in s.N0:
            if decision_variables[i,j,k] == 1:
                print(f'Job {j} scheduled in sequence of job {k} in machine {i}')

for i in s.N:
    print(f'Job {i} completed at {completion_times[i]}s')

print(f'The makespan has been minimized to {maximum_makespan}s')

for i in s.M:
    for k in s.N:
        if assignments[i,k] == 1:
            print(f'Job {k} has been assigned to machine {i}')

Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2519400
Academic license 2519400 - for non-commercial use only - registered to gi___@studenti.units.it
Job 6 scheduled in sequence of job 9 in machine 1
Job 9 scheduled in sequence of job 25 in machine 1
Job 19 scheduled in sequence of job 6 in machine 1
Job 25 scheduled in sequence of job 0 in machine 1
Job 1 scheduled in sequence of job 0 in machine 2
Job 2 scheduled in sequence of job 1 in machine 2
Job 8 scheduled in sequence of job 11 in machine 2
Job 11 scheduled in sequence of job 17 in machine 2
Job 14 scheduled in sequence of job 8 in machine 2
Job 17 scheduled in sequence of job 2 in machine 2
Job 7 scheduled in sequence of job 0 in machine 3
Job 12 scheduled in sequence of job 16 in machine 3
Job 15 scheduled in sequence of job 7 in machine 3
Job 16 scheduled in sequence of job 15 in machine 3
Job 18 scheduled in sequence of job 30 in machine 3
Job 20 scheduled in sequence of job 18 in machine

# Mathematical Programming Algorithm (MPA)

The paper cites a decomposition algorithm (Branch-and-check) based on the previous formulation.

It is an exact formulation based on a master algorithm and a sequence algorithm:
- The master algorithm is responsible of assigning jobs to machines
- The sequence problem is responsible of finding the optimal sequence of jobs for each machine

In [5]:
# TODO: fix this part

s = MPA(execution_times = P_dict, setup_times = S_dict)
decision_variables,maximum_makespan = s.solve(options=options)
for i in s.M:
    for j in s.N:
        for k in s.N0:
            if decision_variables[i,j,k] == 1:
                print(f'Job {j} scheduled in sequence of job {k} in machine {i}')

print(f'The makespan has been minimized to {maximum_makespan}s')

# Fix-and-Optimize approach

In [6]:
greedy = Greedy(execution_times=P_dict, setup_times=S_dict)
solution = greedy.solve()

for i in s.M:
    for j in s.N0:
        for k in s.N0:
            if solution[i,j,k] == 1:
                print(f'Job {j} scheduled in sequence of job {k} in machine {i}')

for i in s.M:
    for k in s.N:
        if assignments[i,k] == 1:
            print(f'Job {k} has been assigned to machine {i}')

for j in s.N:
    print(f'Job {j} completed at {completion_times[j]}s')

Job 0 scheduled in sequence of job 20 in machine 1
Job 6 scheduled in sequence of job 0 in machine 1
Job 7 scheduled in sequence of job 6 in machine 1
Job 8 scheduled in sequence of job 7 in machine 1
Job 11 scheduled in sequence of job 8 in machine 1
Job 20 scheduled in sequence of job 11 in machine 1
Job 0 scheduled in sequence of job 15 in machine 2
Job 2 scheduled in sequence of job 14 in machine 2
Job 14 scheduled in sequence of job 0 in machine 2
Job 15 scheduled in sequence of job 29 in machine 2
Job 21 scheduled in sequence of job 28 in machine 2
Job 22 scheduled in sequence of job 2 in machine 2
Job 28 scheduled in sequence of job 22 in machine 2
Job 29 scheduled in sequence of job 21 in machine 2
Job 0 scheduled in sequence of job 30 in machine 3
Job 4 scheduled in sequence of job 26 in machine 3
Job 10 scheduled in sequence of job 0 in machine 3
Job 19 scheduled in sequence of job 25 in machine 3
Job 25 scheduled in sequence of job 4 in machine 3
Job 26 scheduled in sequence

In [7]:
m = FixOpt(initial_solution=solution, setup_times=S_dict,execution_times=P_dict, N=N, M=M, N0=N0, 
           subproblem_size_adjust_rate=0.1, t_max = 30, subproblem_runtime_limit=10, subproblem_size=5)

solution, makespan = m.solve()
clear_output(wait=True)
for i in M:
    for j in N0:
        for k in N0:
            if solution[i,j,k] == 1:
                print(f'Job {j} scheduled after job {k} in machine {i}')
print(f'The makespan has been minimized to {makespan}s')

Job 0 scheduled after job 10 in machine 1
Job 6 scheduled after job 9 in machine 1
Job 9 scheduled after job 0 in machine 1
Job 10 scheduled after job 19 in machine 1
Job 19 scheduled after job 6 in machine 1
Job 0 scheduled after job 14 in machine 2
Job 2 scheduled after job 22 in machine 2
Job 8 scheduled after job 2 in machine 2
Job 14 scheduled after job 17 in machine 2
Job 17 scheduled after job 21 in machine 2
Job 21 scheduled after job 8 in machine 2
Job 22 scheduled after job 0 in machine 2
Job 0 scheduled after job 7 in machine 3
Job 7 scheduled after job 30 in machine 3
Job 12 scheduled after job 16 in machine 3
Job 15 scheduled after job 0 in machine 3
Job 16 scheduled after job 26 in machine 3
Job 18 scheduled after job 15 in machine 3
Job 26 scheduled after job 18 in machine 3
Job 30 scheduled after job 12 in machine 3
Job 0 scheduled after job 23 in machine 4
Job 3 scheduled after job 20 in machine 4
Job 20 scheduled after job 24 in machine 4
Job 23 scheduled after job 3 